In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('../data/userData.csv')

df.drop(['weekday'], axis=1, inplace=True)
df

,food_name,time_of_day
0,sweet tea,Night
1,chick-fil-a sandwich,Night
2,chick-fil-a sandwich,Night
3,chick-fil-a 8 count nugget,Night
4,chocolate chip muffin,Night
...,...,...
359,optimum nutrition scoop of whey,Evening
360,cups of milk,Evening
361,bags of oatmeal,Night
362,optimum nutrition shake with 2 cups of milk,Evening


In [5]:
from sklearn.preprocessing import LabelEncoder

data_encoded = pd.get_dummies(df[['time_of_day']]) # time of day is Night, Morning, Afternoon, Evening
data_encoded = pd.concat([df[['food_name']], data_encoded], axis=1) # food_name is a string 

y = df['food_name'].values

encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

data_encoded['encoded_y'] = encoded_y

data_encoded.sort_values('food_name', inplace=True)

data_encoded

,food_name,time_of_day_Afternoon,time_of_day_Evening,time_of_day_Morning,time_of_day_Night,encoded_y
57,12 count meal from chick-fil-a,False,False,False,True,0
114,2 percent fairlife chocolate milk,False,False,False,True,1
135,2 pieces of toast with jelly and butter,False,False,False,True,2
18,3 chocolate doughnuts,False,True,False,False,3
357,3x3 from in-n-out,False,False,False,True,4
...,...,...,...,...,...,...
249,whataburger fries,False,True,False,False,136
250,whataburger sweet tea,False,True,False,False,137
60,yogurt,False,False,False,True,138
306,yogurt,False,False,False,True,138


In [6]:
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [58]:
y = data_encoded['encoded_y'].values
X = data_encoded.drop(['food_name', 'encoded_y'], axis=1).values

print(tf.__version__)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_train = X
y_train = y

num_classes = len(np.unique(y))
print(num_classes) # 139

model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,), name='input'),
    tf.keras.layers.Dense(16, activation='relu', name='hidden1'),
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2.16.1
139


/usr/local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# model.evaluate(X_test, y_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.1579 - loss: 5.2205


[5.220489025115967, 0.15789473056793213]

In [60]:
# model.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))
model.fit(X_train, y_train, batch_size=32, epochs=30)

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1855 - loss: 3.4915 
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2173 - loss: 3.4900 
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2224 - loss: 3.4667 
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2313 - loss: 3.3807 
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2106 - loss: 3.2873 
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2029 - loss: 3.4313 
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2251 - loss: 3.4044 
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2163 - loss: 3.4301 
Epoch 9/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2356 - loss: 3.3670 
Epoch 10/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1994 - loss: 3.4295 
Epoch 11/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2086 - loss: 3.4013 
Epoch 12/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 

In [61]:
permutations_day = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.bool_)


predictions = model.predict(permutations_day)

top_n = 5

top_n_indices = np.argsort(predictions, axis=1)[:, -top_n:][:, ::-1]
top_n_probabilities = np.sort(predictions, axis=1)[:, -top_n:][:, ::-1]


top_n_indices, top_n_probabilities

flattened =  encoder.inverse_transform(top_n_indices.flatten())

top_n_labels = flattened.reshape(top_n_indices.shape)

top_n_labels, top_n_probabilities

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


(array([['cups of milk', 'protein shake with milk and peanut butter',
         'buccees beef jerky', 'fairlife chocolate milk',
         'optimum nutrition protein shake'],
        ['chick-fil-a sandwich', 'sweet tea', 'kodiak cake waffles',
         'chipotle chicken and queso bowl', 'chick-fil-a granola parfait'],
        ['sweet tea', 'fairlife chocolate milk',
         'venti caramel macchiato', 'chick-fil-a sandwich', 'cashews'],
        ['chick-fil-a sandwich', 'sweet tea', 'banana', 'milk',
         'chick-fil-a granola parfait']], dtype=object),
 array([[0.08543102, 0.07360816, 0.04091068, 0.04045256, 0.0391572 ],
        [0.12379178, 0.10794039, 0.03872909, 0.023019  , 0.02225682],
        [0.10100175, 0.08532845, 0.07692828, 0.07160633, 0.06723933],
        [0.26899245, 0.21906525, 0.02857831, 0.01728279, 0.0168729 ]],
       dtype=float32))

In [62]:
time_of_day_categories = ['Morning', 'Afternoon', 'Evening', 'Night']

with open('../out/food_recommendations.txt', 'w') as f:
    # Iterate over time of day categories
    for time_of_day, predictions, probabilities in zip(time_of_day_categories, top_n_labels, top_n_probabilities):
        # Write time of day
        f.write("Time of Day: {}\n".format(time_of_day))
        f.write("Predictions:\n")
        # Iterate over predictions and probabilities
        for i, (prediction, probability) in enumerate(zip(predictions, probabilities), 1):
            # Write each prediction and its probability
            f.write("    - Prediction {}: {}, Probability: {:.2f}%\n".format(i, prediction, probability * 100))
        # Add a newline for better readability between time of day categories
        f.write("\n")
